## Времянные ряды 4

In [46]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd 
import seaborn as sns
import statsmodels.formula.api as smf
import statsmodels.api as sm
import warnings
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import f_regression, mutual_info_regression, SelectKBest, SelectPercentile, SelectFromModel

import os

__1. Прочитайте базу my_PJME_MW.csv и сделайте ее перерасчет (ресемплинг) в представление по неделям.__

In [47]:
data = pd.read_csv('../lesson4/my_PJME_MW.csv', index_col=[0], parse_dates=[0])
data_w = data.resample('W').mean()
data_w.drop(columns=['year', 'hour','dayofweek','dayofmonth', 'dayofyear',
                     'is_holiday', 'is_prev_holiday', 'is_next_holiday'], axis=1,inplace=True)

In [48]:
data_w.head()

,PJME_MW,month,quarter,weekofyear
Date,,,,
2002-01-06,32392.160839,1.000000,1.0,1.0
2002-01-13,31487.386905,1.000000,1.0,2.0
2002-01-20,31452.880952,1.000000,1.0,3.0
2002-01-27,30141.529762,1.000000,1.0,4.0
2002-02-03,29196.488095,1.428571,1.0,5.0


__2. Постройте модель предсказания 4-й точки от текущей (h = 4), используя результаты автокорреляционного анализа из предшествующих уроков.__

In [49]:
first_diff =np.array(data_w.iloc[1:,0].values - data_w.iloc[:-1,0].values).astype('float32') 
data_w['diff1'] = 0.0
data_w['diff1'].values[1:] = first_diff

In [50]:
p = 4
for i in range(p):
    data_w['t'+str(p-i)]=0
    data_w['t'+str(p-i)].values[p:] = data_w['PJME_MW'].values[i:-(p)+i]
    

for i in range(p):
    data_w['d'+str(p-i)]=0
    data_w['d'+str(p-i)].values[p:] = data_w['diff1'].values[i:-(p)+i]    

In [51]:
scl_target=StandardScaler() 
h=4
target = data_w['PJME_MW'].values[h+p:].reshape((data_w.shape[0]-h-p,1))
y = scl_target.fit_transform(target)
y.shape

(858, 1)

In [52]:
scl_x=StandardScaler() 

X = data_w.iloc[p:-h,:].values
X = scl_x.fit_transform(X.astype('float64'))
X.shape

(858, 13)

In [53]:
f_test, _ = f_regression(X, y.ravel())
f_test /= np.max(f_test)

sfk =SelectKBest(f_regression, k=5)
X_new1 =sfk.fit_transform(X, y.ravel())

sfp=SelectPercentile(f_regression, percentile=10)
X_new2 = sfp.fit_transform(X, y.ravel())

gr_model = GradientBoostingRegressor(max_depth=5, random_state=40, n_estimators=100).fit(X, y.ravel())
sfm = SelectFromModel(gr_model,  max_features = 5)

sfm.fit(X, y.ravel())
X_new3 = sfm.transform(X)


In [54]:
list_param_k =np.where(sfk.get_support())[0]
list_param_p =np.where(sfp.get_support())[0]
list_param_m =np.where(sfm.get_support())[0]

print('Номера признаков k лучших признаков : ',list_param_k)
print('Номера признаков лучших признаков по процентной форме: ',list_param_p)
print('Номера признаков для выбора по модели : ',list_param_m)

Номера признаков k лучших признаков :  [ 0  8 10 11 12]
Номера признаков лучших признаков по процентной форме:  [0 8]
Номера признаков для выбора по модели :  [0 1 3 5]


In [55]:
list_par=[0,1,3,5,8,10,11,12]
data_w.columns[list_par]

Index(['PJME_MW', 'month', 'weekofyear', 't4', 't1', 'd3', 'd2', 'd1'], dtype='object')

In [56]:
X_new = X[:,list_par]

In [57]:
def split_data( data, split_date ):
    return data.loc[data.index.get_level_values('Date') <= split_date].copy(), \
           data.loc[data.index.get_level_values('Date') >  split_date].copy()

In [58]:
train, test = split_data( data_w, '01-06-2016')
X_train =X_new[:train.shape[0],:]
y_train =y[:train.shape[0],0]
X_test =X_new[train.shape[0]:,:]
y_test =y[train.shape[0]:,0]

In [59]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [60]:
param_dist = {"max_depth": [3, 2,1,4,5], 
              "n_estimators":[100,200,300,400,500]}

In [61]:
n_iter_search = 10

model_gr =GradientBoostingRegressor()

random_search = RandomizedSearchCV(model_gr, param_distributions=param_dist, n_iter=n_iter_search, cv=5, iid=False)

random_search.fit(X_train, y_train)

report(random_search.cv_results_)


D:\Compile\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:849: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


Model with rank: 1
Mean validation score: 0.642 (std: 0.055)
Parameters: {'n_estimators': 500, 'max_depth': 1}

Model with rank: 2
Mean validation score: 0.637 (std: 0.056)
Parameters: {'n_estimators': 400, 'max_depth': 1}

Model with rank: 3
Mean validation score: 0.623 (std: 0.061)
Parameters: {'n_estimators': 100, 'max_depth': 2}



In [62]:
model_g_best = GradientBoostingRegressor(max_depth=1,  n_estimators=500)
model_g_best.fit( X_train, y_train ) 
y_test_pred_g = model_g_best.predict(X_test)


__3. Постройте модель предсказания 1-й точки от текущей (h = 1), используя результаты автокорреляционного анализа из предшествующих уроков.__

__4. Примените авторекурсию и сравните результат в 4-й точке путем прямого моделирования и путем рекурсивного моделирования на длине выборки из 4-х точек.__

In [63]:
#['PJME_MW', 'month', 'weekofyear', 't4', 't1', 'd3', 'd2', 'd1']
N = 1000
for i in range(N):
    y_test_100[i] = model_g_best.predict(X_test_100[i,:].reshape((1,18)))
    # заполняем вектор для прогноза следующей точки
    X_test[i+1,0] = y_test_pred_g[i]
    X_test[i+1,17] = X_test[i+1,0]*(1-0.08)+ 0.08 * X_test[i,17] 
    X_test[i+1,16] = X_test[i+1,0]*(1-0.07)+ 0.07 * X_test[i,16] 
    if i>p:
        X_test[i+1,17] = np.mean(X_test[i+1-p:i+1,0]) 
        
    
    
plt.figure( figsize=( 15, 7 ))


plt.xlabel('Время')
plt.ylabel('Потребление энергии, МВт')
plt.plot(test.index[1:N+1],y_test_pred_g[:N], 'r',label='predict GB data' )
plt.plot(test.index[1:N+1],y_test[:N], 'g',label='test data', alpha=0.5)
plt.plot(test.index[1:N+1],y_test_100[:N], '--r',label='slide predict data')

plt.legend()
plt.show()

NameError: name 'X_test_100' is not defined

__5. Оцените рост ошибки прогнозирования рекурсивной модели в интервалах от 1-й до 10-й, от 10-й до 20-й, от 20-й до 30-й, .. .10*i-й до (i+1)*10 -й,... ,от 90 до 100-й точках (используйте осреднение по десяткам точек)__

__6. Сделайте вывод о том, как изменилось поведение ошибки предсказания ряда.__